In [3]:
%pip install pandas
%pip install nltk
%pip install sklearn
%pip install scikit-learn

     -------------------------------------- 11.2/11.2 MB 744.4 kB/s eta 0:00:00
     -------------------------------------- 14.8/14.8 MB 630.8 kB/s eta 0:00:00
     ------------------------------------ 341.8/341.8 kB 732.0 kB/s eta 0:00:00
     -------------------------------------- 502.3/502.3 kB 1.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_validation_split

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Initially Viewing the Data

In [4]:
df = pd.read_csv("emails.csv")
df.head()
print(df.shape)


(5172, 3002)


## Preprocessing Pipeline

In [5]:
stopwordList = stopwords.words('english')
columnsToDrop = list(set(stopwordList) & set(df.columns))
df = df.drop(columnsToDrop, axis=1)
df.head()

,Email No.,ect,hou,enron,com,gas,deal,meter,hpl,please,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,24,27,1,3,1,0,0,0,2,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,1,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,22,10,0,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,17,9,0,0,2,0,3,0,1,...,0,0,0,0,0,0,0,1,0,0


In [7]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=69)
print("Train shape: ", train_df.shape)
print("Test shape: ", test_df.shape)
print("Train spam ratio: ", train_df['spam'].mean())
print("Test spam ratio: ", test_df['spam'].mean())
print(train_df.head())
print(test_df.head())

Train shape:  (4137, 2868)
Test shape:  (1035, 2868)
Train spam ratio:  0.016678752719361856
Test spam ratio:  0.022222222222222223
       Email No.  ect  hou  enron  com  gas  deal  meter  hpl  please  ...   
4591  Email 4592    1    0      3    2    0     0      0    0       1  ...  \
4537  Email 4538    1    3      0    1    0     0      0    0       0  ...   
747    Email 748    9    5      7    0    0     3      0    0       1  ...   
4639  Email 4640    1    1      0    0    0     0      0    0       1  ...   
3397  Email 3398    2    0      0    1    0     0      0    0       0  ...   

      connevey  jay  valued  lay  infrastructure  military  allowing  ff  dry   
4591         0    0       0    0               0         0         0   0    0  \
4537         0    0       0    0               0         0         0   1    0   
747          0    0       0    0               0         0         0   0    0   
4639         0    0       0    0               0         0         0   0   